# 2. Tool to identify some components that have caused the electrical events

<p> This jupyter notebook was used to manually identify some of the componennts that have caused the electrical events, that were previously hand-labeled. The components identified are <b> pumps, grinders (motor) and heaters </b>in the coffeemaker. </p> <b> This is the second notebook in the labeling pipeline of CREAM. </b>
<div class="alert alert-info">
    <h3>Instructions for using this notebook</h3>
    <p> In the following, we load the electrical events that have been previously labeled with the "1_electrical_events_labeling_tool.ipynb" notebook. </p>
    <p> Proceed at the end of the notebook with the corresponding cell for the labeling. Follow the instructions given there. </p>


## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import h5py
import pandas as pd
import os
import sys
from pathlib import Path
from datetime import datetime
from datetime import timedelta
import math
import pdb
import scipy
# Add project path to path for import
project_path = os.path.abspath("..")
if project_path not in sys.path:
    sys.path.append(project_path)

# Add module path to path for import
module_path = os.path.abspath("../data_utility/data_utility.py")
if module_path not in sys.path:
    sys.path.append(module_path)
    
from data_utility import CREAM_Day # class to work with a day of the CREAM Dataset

%matplotlib notebook
# Intentional replication is necessary
%matplotlib notebook
%load_ext autoreload
# Reload all modules every time before executing the Python code typed.
%autoreload 2 
# Import some graphical modules
from IPython.display import display, clear_output
from ipywidgets import Button, Layout, ButtonStyle, HBox, VBox, widgets, Output
from IPython.display import SVG, display, clear_output

import subprocess
import glob

## Global Functions

In [ ]:
def plot_event_window(event_timestamp:pd.Timestamp, window_size, current_CREAM_day:CREAM_Day, concurrent_events_dict):
    """
    
    Plots a window of window_size in each direction around the event_timestamp.
    The event_timestamp marks the beginning of the minute where the event stopped.
    So instead of directly using the event_timestamp, we plot the event_timestamp + 59 seconds 
    to mark the end of the minute in that the event stopped.
    Therefore the event has happended before the point that is marked as a bold red line.
    The current signal of the coffee maker is plotted.
    The event type is the label the event gets.
    If a concurrent_events_dict is provided, with the keys being the name of the event list and the values being the event dataframes,
    all other events that happen within the window of interest are also plotted.
    Appliance events are bold orange lines.
    Other events are dashed red lines.

    """
    # Import and set globals necessary for the click functions
    
    global EVENT_TIMESTAMP
    global WINDOW_START_TS
    global COMPONENTS_DF
    
    # Instead of taking the event timestamp directly we take the END of the minute
    end_event_timestamp  = event_timestamp + timedelta(seconds=59)
    
    # Tackle border cases of the timestamp
    if end_event_timestamp - timedelta(seconds=window_size) < current_CREAM_day.minimum_request_timestamp: # in case we are at the beginning of the day
        duration_to_left = end_event_timestamp - current_CREAM_day.minimum_request_timestamp
        duration_to_left = duration_to_left.total_seconds() # amount of data that we load now to the left of the current timestmap
        duration_to_right = window_size #to the right we can load the full window 
        
    elif end_event_timestamp + timedelta(seconds=window_size) >  current_CREAM_day.maximum_request_timestamp: # in case we are at the end of the day
        duration_to_right = current_CREAM_day.maximum_request_timestamp - end_event_timestamp 
        duration_to_right = duration_to_right.total_seconds() #amount of data that we load now to the right of the current timestamp
        duration_to_left = window_size #to the left we can load the full window
    
    else: # if we have anough datapoints to the left and to the right to load the full WINDOW_SIZE in each direction
        duration_to_left = window_size
        duration_to_right = window_size
    
    # Create the start- and end-timestamp and compute the overall duration of the window
    duration = duration_to_left + duration_to_right
    
    start_ts = end_event_timestamp - timedelta(seconds=duration_to_left)
        
    end_ts = end_event_timestamp + timedelta(seconds=duration_to_right)
        
    # Load the data
    
    voltage, current = current_CREAM_day.load_time_frame(start_datetime=start_ts, duration=duration) #and WINDOW_SIZE seconds after the event
        
    # Compute the index of the event, using the timestamp
    end_event_index = current_CREAM_day.get_index_from_timestamp(start_ts, end_event_timestamp)
    
    fig, ax = plt.subplots(1,1)   
    fig.canvas.mpl_connect('button_press_event', onclick) #append event to figure
    
    xticks = np.arange(len(current))
    ax.plot(xticks, current, markersize=0.1, alpha=0.6) 
    
    ax.tick_params(axis='x', rotation=90) #rotate the xlabels

    if np.max(current) < 1: #in case of noise, show an appropriate range
        ax.set_ylim([-6,6])
    
    # Plot the event line
    ax.axvline(end_event_index, color="red", linewidth=1.5)
    
    # Add other events that happend within the window
    if len(concurrent_events_dict) > 0:
        
        for event_list_name, concurrent_events_df in concurrent_events_dict.items():
            
            # If an already refined timestamp list (either product, or maintenance) is provided, one
            # can plot the detailed end timestamps instead of the coarse grained ones that are not refined yet
            if "End_Timestamp" in concurrent_events_df.columns:
                ts_column_name = "End_Timestamp"
            else:
                ts_column_name = "Timestamp"
                
            concurrent_events_df_roi = concurrent_events_df[(concurrent_events_df[ts_column_name] <= end_ts) & (concurrent_events_df[ts_column_name] >= start_ts)]
            
            if len(concurrent_events_df_roi) > 0:
                for i, row in concurrent_events_df_roi.iterrows():
                    
                    # Get the event index
                    i = current_CREAM_day.get_index_from_timestamp(start_ts, row[ts_column_name])

                    # Some plotting adjustments, depending on the type of event that is plotted
                    if "component" in event_list_name:
                        color ="orange"
                        linewidth=1.5
                        
                            
                    else: # in case of product or maintenance events
                        color="red"
                        if "product" in event_list_name:
                            if "Product" in concurrent_events_df_roi.columns:
                                label = row.Product
                            elif "Event_Type" in concurrent_events_df_roi.columns:
                                label= row.Event_Type
                            else:
                                label = "unspecified"
                            
                            linewidth=1.2
                                
                        elif "maintenance" in event_list_name:
                            
                            if "Activity" in concurrent_events_df_roi.columns:
                                label = row.Activity
                            elif "Event_Type" in concurrent_events_df_roi.columns:
                                label= row.Event_Type
                            else:
                                label = "unspecified"
                            linewidth=1.2
                            
                        else:
                            label = "Unknown"
                            linewidth=0.6
                    
                    # Plot the line
                    ax.axvline(i, color=color, linestyle=":", label=label, linewidth=linewidth)
                        
        
    if len(COMPONENTS_DF) > 1:

        # use mask here because of misaligned indices 
        mask = (COMPONENTS_DF.Timestamp <= end_ts) & (COMPONENTS_DF.Timestamp >= start_ts)
        concurrent_events_df_roi = COMPONENTS_DF.loc[mask.values]
        concurrent_events_df_roi = concurrent_events_df_roi[concurrent_events_df_roi.Component!="unlabeled"] #only take the ones with an already labeled component
        if len(concurrent_events_df_roi) > 0:
            for i, row in concurrent_events_df_roi.iterrows():
                i = current_CREAM_day.get_index_from_timestamp(start_ts, row.Timestamp)
                ax.axvline(i, color="green", linestyle=":", label="already labeled end " + str(i))
    
    # add time information to plot
    samples_per_minute = current_CREAM_day.sampling_rate * 60 #every 60 seconds
    
    if len(current) % samples_per_minute  == 0: #just in case the parameters are changed and there are no full minutes in the signals
        step = len(current) / samples_per_minute
        for i in range(0, int(step+1)):
            ax.axvline(i*samples_per_minute, color="black", ymax=0.1)
    
    fig.suptitle("Event :" +  "\n" + str(str(start_ts) + " - " + str(end_ts)))
    ax.legend(loc='upper right')
    
    EVENT_TIMESTAMP = event_timestamp
    WINDOW_START_TS = start_ts
    
    return fig, ax

##  Global Variables

In [ ]:
EVENT_INDEX = int(0) # index of the EVENTS_TO_LABEL_DF that the programm is currently at
EVENTS_TO_LABEL_DF = None # dataframe of the list of events to label
EVENT_TIMESTAMP = None # timestamp of the event that is in the current focus
WINDOW_START_TS = None # start timestamp of the window we are currently looking at
LAST_EVENT_CLICKED_LOC_LIST = [] # list of the locs of the last events clicked
LABELED_TIMESTAMP = None # the labeled timestamp
WINDOW_SIZE = int(120) # seconds, the window size in each direction around and event to be displayed
ALL_DAYS = ["2018-08-23" , "2018-08-24" , "2018-08-25",  "2018-08-26" , "2018-08-27" , "2018-08-28" ,
"2018-08-29", "2018-08-30", "2018-08-31", "2018-09-01", "2018-09-02" , "2018-09-03" ,  "2018-09-04" ,
"2018-09-05", "2018-09-06", "2018-09-07", "2018-09-08" , "2018-09-09" , "2018-09-10", "2018-09-11", "2018-09-12" 
"2018-09-13" ,"2018-09-14" ,"2018-09-15" ,  "2018-09-16", "2018-09-17", "2018-09-18","2018-09-19"  , "2018-09-20" ,
"2018-09-21" , "2018-09-22" ,  "2018-09-23" ,"2018-09-24" ,"2018-09-25" ,"2018-09-26" , "2018-09-27", "2018-09-28" ,
"2018-09-29" , "2018-09-30" , "2018-10-01" ,"2018-10-02" , "2018-10-03" ,"2018-10-04", "2018-10-05" , "2018-10-06" ,
"2018-10-07", "2018-10-08" ]

## Widget functions for the UI

In [ ]:
closest_event_loc = None
timestamp_clicked = None

In [ ]:
def onclick(event):
    """
    Function to be executed in case of a click event at a figure.
    """
    global COMPONENTS_DF # Dataframe containing the component events
    global COMPONENT_NAME # Name of the component currently labeled
    global LAST_EVENT_CLICKED_LOC_LIST # list of locs of the last events clicked, used for deleting last click in case of errors
    global EVENT_TIMESTAMP #timestamp of the event of interest that was autoamticcaly generated
    global WINDOW_START_TS #start timestamp of the window we are currently looking at
    global current_CREAM_day #object representing the current day in the CREAM dataset
    global EVENT_INDEX # index of the EVENTS_TO_LABEL_DF that the programm is currently at
    
    global closest_event_loc
    global timestamp_clicked
    # Take the event index from the click, convert it to a timestamp
    timestamp_clicked = current_CREAM_day.get_timestamp_from_index(WINDOW_START_TS, math.floor(event.xdata))
    
    if  timestamp_clicked > EVENT_TIMESTAMP + timedelta(seconds=60):
        print("The red timestamp is generated after the event is completed! Hence, do not place the click after it!")
        return
   
    
   
    event_before = COMPONENTS_DF[COMPONENTS_DF.Timestamp <= timestamp_clicked].iloc[-1]
    event_after = COMPONENTS_DF[COMPONENTS_DF.Timestamp > timestamp_clicked].iloc[0]
    delta_before = timestamp_clicked - event_before.Timestamp
    delta_before = delta_before.total_seconds()
    delta_after = event_after.Timestamp - timestamp_clicked 
    delta_after = delta_after.total_seconds()
    if delta_before <= delta_after:
        closest_event_loc = event_before.name
    else:
        closest_event_loc = event_after.name
    COMPONENTS_DF.at[closest_event_loc, "Component"] = COMPONENT_NAME
    
    # Store the loc to enable the delete function in case of errors
    LAST_EVENT_CLICKED_LOC_LIST.append(closest_event_loc)
    
    # Increment the index we are currently looking at
    EVENT_INDEX += 1 
    

    
    return
    

In [ ]:
def display_initial_event(event_index_p=0):
    """
    Display the start event. This is set to 0 as per default!
    In case of interruptions in the labeling process or in case of errors, you can restart labeling at
    an arbitrary index using the event_index_p paramter.
    """
    
    global COMPONENTS_DF # Dataframe containing the component events
    global COMPONENT_NAME # Name of the component currently labeled
    global LAST_EVENT_CLICKED_LOC_LIST # loc of the last event clicked, used for deleting last click in case of errors
    global CONCURRENT_EVENTS_DICT # dictionary containg the events happening concurrently, used for plotting
    global EVENTS_TO_LABEL_DF # dataframe of the list of events to label
    global EVENT_INDEX # event index we are currently processing
    global FIG # global figure object
    global AX # global axis object
    global current_CREAM_day # global CREAM_day object
    global WINDOW_SIZE # global WINDOW_SIZE
        
    plt.clf()
    clear_output()
    
    if EVENT_INDEX > len(EVENTS_TO_LABEL_DF)-1:
        print("THIS WAS THE LAST EVENT! YOU ARE DONE!")
        return 
    
    
    # For the timestamp we need to check if we need to create the corresponding CREAM_Day object, or if it already exists
    event_timestamp = EVENTS_TO_LABEL_DF.iloc[EVENT_INDEX].Timestamp
    event_date = str(EVENTS_TO_LABEL_DF.iloc[EVENT_INDEX].Date)
    
    if current_CREAM_day.day_date != event_date: # if the event does not lie withing the current CREAM_day object, create a new one
        day_path = os.path.join(PATH_TO_DATA, event_date) 
        current_CREAM_day = CREAM_Day(cream_day_location=day_path,use_buffer=True, buffer_size_files=2) 
        
    FIG, AX = plot_event_window(event_timestamp = EVENTS_TO_LABEL_DF.iloc[EVENT_INDEX].Timestamp, 
            window_size = WINDOW_SIZE,
            current_CREAM_day = current_CREAM_day,
            concurrent_events_dict = CONCURRENT_EVENTS_DICT)
        
    FIG.show()
    display(button_box)

In [ ]:
def on_next_clicked(event):
  
    
    global LABEL_DESTINATION_PATH #location where the event labels will be stored, is user specified
    global WINDOW_START_TS #start timestamp of the window we are currently looking at
    global COMPONENTS_DF # Dataframe containing the component events
    global COMPONENT_NAME # Name of the component currently labeled
    global LAST_EVENT_CLICKED_LOC_LIST # loc of the last event clicked, used for deleting last click in case of errors
    global CONCURRENT_EVENTS_DICT # dictionary containg the events happening concurrently, used for plotting
    global EVENTS_TO_LABEL_DF # dataframe of the list of events to label
    global EVENT_INDEX # event index we are currently processing
    global FIG # global figure object
    global AX # global axis object
    global current_CREAM_day # global CREAM_day object
    global WINDOW_SIZE # global WINDOW_SIZE
    
 
    save_labels(destination=LABEL_DESTINATION_PATH) #save it
    
    plt.clf()
    clear_output()
    
    
    if EVENT_INDEX > len(EVENTS_TO_LABEL_DF)-1:
        print("THIS WAS THE LAST EVENT! YOU ARE DONE!")
        return 

    print("This is event number " + str(EVENT_INDEX) + " of " + str(len(EVENTS_TO_LABEL_DF)))
    
    
    # For the timestamp we need to check if we need to create the corresponding CREAM_Day object, or if it already exists
    event_timestamp = EVENTS_TO_LABEL_DF.iloc[EVENT_INDEX].Timestamp
    event_date = str(EVENTS_TO_LABEL_DF.iloc[EVENT_INDEX].Date)
    if current_CREAM_day.day_date != event_date: # if the event does not lie withing the current CREAM_day object, create a new one
        day_path = os.path.join(PATH_TO_DATA, event_date) 
        current_CREAM_day = CREAM_Day(cream_day_location=day_path,use_buffer=True, buffer_size_files=2) 
        
    FIG, AX = plot_event_window(event_timestamp = EVENTS_TO_LABEL_DF.iloc[EVENT_INDEX].Timestamp, 
        window_size = WINDOW_SIZE,
        current_CREAM_day = current_CREAM_day,
        concurrent_events_dict = CONCURRENT_EVENTS_DICT)
  
    FIG.show()
    display(button_box)
       

In [ ]:
def save_labels(destination: str):
    global EVENT_INDEX
    global COMPONENTS_DF
    global COMPONENT_NAME
    
    filename = "labeled_component_events.csv" 
    
    if EVENT_INDEX % 10 == 0 and EVENT_INDEX > 0: #every 10 events: before storing the new file, save the old one       
        os.rename(os.path.join(destination, filename), os.path.join(destination, "previous_component_event_labels.csv"))
    
    #Store the new one
    COMPONENTS_DF.to_csv(os.path.join(destination, filename), index=False)

In [ ]:
def on_delete_clicked(event):
    """
    Deletes the last click from every key in the event_dictionary and returns to the previous window
    """
    global COMPONENTS_DF # Dataframe containing the component events
    global COMPONENT_NAME # Name of the component currently labeled
    global LAST_EVENT_CLICKED_LOC_LIST # loc of the last event clicked, used for deleting last click in case of errors
    global CONCURRENT_EVENTS_DICT # dictionary containg the events happening concurrently, used for plotting
    global EVENTS_TO_LABEL_DF # dataframe of the list of events to label
    global EVENT_INDEX # event index we are currently processing
    global FIG # global figure object
    global AX # global axis object
    
    if EVENT_INDEX <= 0 or LAST_EVENT_CLICKED_LOC_LIST is None: #we arrived at the first event again
        print("This is the first event, you can not go further back in time!")
        return 
    
    COMPONENTS_DF.at[LAST_EVENT_CLICKED_LOC_LIST[EVENT_INDEX], "Component"] = "unlabeled"
    
    
    EVENT_INDEX = EVENT_INDEX - 1 # adjust EVENT_INDEX
    
    FIG, AX = plot_event_window(event_timestamp = EVENTS_TO_LABEL_DF[EVENT_INDEX].Timestamp, 
            window_size = WINDOW_SIZE,
            current_CREAM_day = current_CREAM_day,
            concurrent_events_dict = CONCURRENT_EVENTS_DICT)
    
    # Now display the previous event
    plt.clf()
    clear_output()
    print("The current Event Index is " + str(EVENT_INDEX))
    FIG.show()
    display(button_box)
    
    return EVENT_DICTIONARY

# Only touch this area in the notebook to alter variables like, for example, the path to the dataset

<div class="alert alert-danger">
    <h3>//ToDo</h3>'
    <p>Please specify the component name to label. </p>
</div>

In [ ]:
COMPONENT_NAME = "millingplant" # 'pump', 'heater'

<div class="alert alert-danger">
    <h3>//ToDo</h3>
    <p>Please specify the path to the main-folder of "CREAM". </p>
</div>

In [ ]:
PATH_TO_DATA = os.path.abspath(os.path.join("..", "..", "Datasets", "CREAM", "CREAM"))

<div class="alert alert-danger">
    <h3>//ToDo</h3>
    <p>Please specify the path to location where you want to store the labels. </p>
</div>

In [ ]:
LABEL_DESTINATION_PATH = os.path.abspath(os.path.join("..", "..", "Datasets", "CREAM", "tmp")) 

## Execute this cell to load the raw electrical events

<p> In the following, we load the electrical events that have been previously labeled with the "1_electrical_events_labeling_tool.ipynb" notebook. </p>
<p> Furthermore, we load the raw product and maintenance events, that contain the timestamps with a per minute precision </p>

In [ ]:
#necessary for the plotting
# Load the events 
day_path = os.path.join(PATH_TO_DATA, "2018-08-24") #arbitrary day to initialize the object
current_CREAM_day = CREAM_Day(cream_day_location=day_path,use_buffer=True, buffer_size_files=2) 

# Load the electrical component events (the raw ones)
COMPONENTS_DF = current_CREAM_day.load_component_events(os.path.join(PATH_TO_DATA, "raw_coffee_maker_logs", "raw_component_events.csv"), raw_file=True, filter_day=False)

# Load the product and the maintenance events (the raw ones, per minute events) and filter for the day
all_maintenance_events = current_CREAM_day.load_machine_events(os.path.join(PATH_TO_DATA, "raw_coffee_maker_logs", "raw_maintenance_events.csv"), raw_file=True, filter_day=False)
all_product_events = current_CREAM_day.load_machine_events(os.path.join(PATH_TO_DATA, "raw_coffee_maker_logs", "raw_product_events.csv"), raw_file=True, filter_day=False)

# Initalize the dictionary that is used to determine concurrent_events in the plot method
CONCURRENT_EVENTS_DICT = {"product_events" : all_product_events, "maintenance_events" : all_maintenance_events}

## Execute this cell to add the "Component" column to the raw_component events from labeling step 1

In [ ]:
if "Component" not in COMPONENTS_DF.columns: #only if the column has not been created before
    COMPONENTS_DF["Component"] = "unlabeled"

# Execute this cell to start the labeling

<p> Click into the figure as close as possible to the event you want to label. The closest event to your click
is then labeled accordingly. </p>
<p> To ease labeling and to raise awareness for concurrent events the follwoing lines are displayed: </p>
    <p> Appliance event labels are shown in dashed orange lines </p>
    <p> Any other product or maintenance event is show with a dashed red line </p>
<p> <b> The red line marks the point by that the event has to be finished latest! </b> </p>
<p> The short black lines represent one minute steps </p>
<p> If you think you are done with this event, click the green <b> "next" </b> button to load the next event and save the previous one </p>
<p> If you have selected <b> "next" </b> accidentially or still to remove the event you have labeled from the previous event, select the red <b> "delete last entry" </b >button </p>

<div class="alert alert-info">
 <h4>Empty Figure or not in interactive mode</h4>
    <p>If the plot does not load or is not in the interactive mode, reexecute the cell or reexcute the import cell</p>
</div>

<div class="alert alert-danger">
    <h3> Do not use the zoom and other capabilities from the plot toolbar</h3>
    <p>Clicks when zooming etc. also get registred as clicks for labels!</p>
</div>

In [ ]:

if COMPONENT_NAME == "millingplant":
    #build the events_to_label and the concurrent_events dict (schauen ob das schon gefilterted erwartet wird!)
    EVENTS_TO_LABEL_DF = None # dataframe of the list of events to label
    
    EVENTS_TO_LABEL_DF = all_maintenance_events[(all_maintenance_events.Activity == 'MillingPlantEspresso') |
                                    (all_maintenance_events.Activity == 'MillingPlantCoffee')]
    
    # sample a random subset, because there are a lot of them
    np.random.seed(42)
    sample_size = int(len(EVENTS_TO_LABEL_DF) * 0.15)
    events_to_label_subset = np.random.choice(EVENTS_TO_LABEL_DF.index, sample_size, replace=False) 
    EVENTS_TO_LABEL_DF = EVENTS_TO_LABEL_DF.loc[events_to_label_subset]
    EVENTS_TO_LABEL_DF.sort_index(inplace=True) #sort by index
    print("Proceed with the labeleling of the millingplant events below!")
    # Create and register Buttons
    next_button =  Button(description="Next -> ",style=ButtonStyle(button_color='green'))
    delete_button = Button(description=" <- Delete last entry",style=ButtonStyle(button_color='red'))
    button_box = HBox([next_button, delete_button])
    next_button.on_click(on_next_clicked)
    delete_button.on_click(on_delete_clicked)
    # Display first event --> event_index is set to zero for the start
    # In case of erros or interruptions, provide another event index to the display_initial_event function
    display_initial_event(event_index_p=0)
    
elif COMPONENT_NAME == "pump":
    
    EVENTS_TO_LABEL_DF = all_product_events[all_product_events.Product == 'hot_water']
    EVENTS_TO_LABEL_DF.sort_index(inplace=True) #sort by index

    print("Proceed with the labeling of the pump events below!")
    # Create and register Buttons
    next_button =  Button(description="Next -> ",style=ButtonStyle(button_color='green'))
    delete_button = Button(description=" <- Delete last entry",style=ButtonStyle(button_color='red'))
    button_box = HBox([next_button, delete_button])
    next_button.on_click(on_next_clicked)
    delete_button.on_click(on_delete_clicked)
    # Display first event --> event_index is set to zero for the start
    # In case of erros or interruptions, provide another event index to the display_initial_event function
    display_initial_event(event_index_p=0)
    
elif COMPONENT_NAME == "heater":
    
    # Simply select all the events on saturdays to be heater events. we only label the on-events
    # We have investigated the data (product events) and no other events can be found on saturdays

    # Get the Saturday dates
    day_information_df = current_CREAM_day.get_weekday_information(date=ALL_DAYS)
    saturdays = day_information_df[day_information_df.Weekday == "Saturday"].Date.values
    
    # Filter for the On-Events and the saturdays in the component events
    mask = (COMPONENTS_DF.Event_Type == "On") & (COMPONENTS_DF.Date.isin(saturdays))
    COMPONENTS_DF.at[mask, "Component"] = "heater"
    
    # To signal that everything is finished
    EVENTS_TO_LABEL_DF = []
    
    print("The heating events have been labeled and saved!")
else:
    raise ValueError("Component name is not available! Please use either millingplant, heater or pump")